In [6]:
import onnx
from onnx import shape_inference
import sys
from tabulate import tabulate
from onnx import onnx_ml_pb2 as xpb2

onnx_model = onnx.load("mobilenetv2-10.onnx", load_external_data=False)
onnx.checker.check_model(onnx_model)

#print(onnx_model)

inferred_model = shape_inference.infer_shapes(onnx_model)

print('shape inference complete ...')

def _parse_element_(elem: xpb2.ValueInfoProto):
    name = getattr(elem, 'name', "None")
    data_type = "NA"
    shape_str = "NA"
    etype = getattr(elem, 'type', False)
    if etype:
        ttype = getattr(etype, 'tensor_type', False)
        if ttype:
            data_type = getattr(ttype, 'elem_type', 0)
            shape = getattr(elem.type.tensor_type, "shape", False)
            if shape:
                shape_str = "["
                dims = getattr(shape, 'dim', [])
                for dim in dims:
                    vals = getattr(dim, 'dim_value', "?")
                    shape_str += (str(vals) + ",")
                shape_str = shape_str.rstrip(",")
                shape_str += "]"
    return name, data_type, shape_str
def _parse_element(elem: xpb2.ValueInfoProto):
    name = getattr(elem, 'name', "None")
    shape_str = "NA"
    etype = getattr(elem, 'type', False)
    if etype:
        ttype = getattr(etype, 'tensor_type', False)
        if ttype:
            shape = getattr(elem.type.tensor_type, "shape", False)
            if shape:
                shape_str = "["
                dims = getattr(shape, 'dim', [])
                for dim in dims:
                    vals = getattr(dim, 'dim_value', "?")
                    shape_str += (str(vals) + ",")
                shape_str = shape_str.rstrip(",")
                shape_str += "]"
    return name, shape_str
def find_operator(graph: xpb2.GraphProto):
    try:
        for i, node in enumerate(inferred_model.graph.node):
                if node.name == "":
                    inferred_model.graph.node[i].name = str(i)
        if type(graph) is not xpb2.GraphProto:
            sys.exit('The input graph is not a GraphProto!')

        node_nList = [k.name for k in graph.node]
        op_dict = {}
        for node in graph.node:
            if node.op_type in op_dict:
                op_dict[node.op_type][node.name] = node_nList.index(node.name)
            else:
                op_dict[node.op_type] = {
                    node.name: node_nList.index(node.name)
                } 

        # init the list
        list_of_layer = []
        list_of_unknown_tensor = []

        # get the occur #, total data element
        for op_type in op_dict:
            occur_num = len(op_dict[op_type])
            total_data_elem = int(0)
            unknown_tensor_list = []

            for op in op_dict[op_type]:
                op_data_elem = int(0)
                for input_name in graph.node[op_dict[op_type][op]].input:
                    input_nlist = [k.name for k in graph.input]
                    initializer_nlist = [k.name for k in graph.initializer]
                    value_info_nlist = [k.name for k in graph.value_info]
                    output_nlist = [k.name for k in graph.output]

                    # get tensor data
                    if input_name in input_nlist:
                        idx = input_nlist.index(input_name)
                        proto= graph.input[idx]
                        data_type = int(1)
                    elif input_name in value_info_nlist:
                        idx = value_info_nlist.index(input_name)
                        proto= graph.value_info[idx]
                        data_type = int(2)
                    elif input_name in initializer_nlist:
                        idx = initializer_nlist.index(input_name)
                        proto= graph.initializer[idx]
                        data_type = int(3)
                    else:
                        print("Can't find the tensor: ", input_name)
                        print('input_nlist:\n', input_nlist)
                        print('===================')
                        print('value_info_nlist:\n', value_info_nlist)
                        print('===================')
                        print('initializer_nlist:\n', initializer_nlist)
                        print('===================')
                        print('output_nlist:\n', output_nlist)
                        print('===================')
                    if proto:
                        input_tensor_size = int(1)
                        if data_type == 1 or data_type == 2:
                            name, shape_str = _parse_element(proto)
                            
                            if(len(shape_str)>2):#不是只有[]
                                shape_str = shape_str.strip('[]')
                                shape_str = shape_str.split(',')
                                shape = []
                                for dim in shape_str:
                                    input_tensor_size *=int(dim)
                        elif data_type == 3:
                            # get the shape of the tensor
                            shape = getattr(proto, 'dims', [])
                            for dim in shape:
                                input_tensor_size *= dim
                        else:
                            print(
                                '[unexpected error] in [get_info] add_up_total_data_elem'
                            )
                    else:
                        print("Can't find the input ", input_name, " of the operator ",
                            op, 'SKIP IT !')
                        unknown_tensor_list.append(
                            (op, input_name, graph.node[op_dict[op_type][op]].op_type))
                        continue

                    op_data_elem += input_tensor_size

                total_data_elem += op_data_elem
            list_of_layer.append((op_type, occur_num, total_data_elem))
            list_of_unknown_tensor.extend(unknown_tensor_list)
        print('list_of_layer BUILT ! ')

        # resort the list
        list_of_layer = sorted(list_of_layer,
                                key=lambda Layer: Layer[2],
                                reverse=True)
        print('list_of_layer RESORTED ! ')

        # Display result
        columns = ['op_type', 'occur #', 'Total data elements #']
        print(tabulate(list_of_layer, headers=columns))
        print(
            '====================================================================================\n'
        )

        columns = ['op_name', 'unfound_tensor', 'op_type']
        print(tabulate(list_of_unknown_tensor, headers=columns))
        print(
            '====================================================================================\n'
        )

        print('DISPLAY successfully ! ')

    except Exception as e:
        print("Unable to display: " + str(e))
        return False

    return True
def get_attribute(graph: xpb2.GraphProto):
    print("attribution\n")
    try:
        conv_attr = []
        for i, node in enumerate(inferred_model.graph.node):
                if node.name == "":
                    inferred_model.graph.node[i].name = str(i)
        # get the idx_list
        # idx_list = get_op_type_name_and_idx(graph)
        node_nlist = [k.name for k in graph.node]
        idx_list = {}
        for node in graph.node:
            if node.op_type in idx_list:
                idx_list[node.op_type][node.name] = node_nlist.index(node.name)
            else:
                idx_list[node.op_type] = {
                    node.name: node_nlist.index(node.name)
                }
        # traverse the idx_list['Conv']
        if 'Conv' not in idx_list.keys():
            print('[ERROR MASSAGE] This graph has no operators "Conv" !')
        else:
            for idx in idx_list['Conv'].values():
                # temp_list = utils._parse_Conv_get_pads_strides(idx, graph)
                temp_list = []
                # print('===========================================================')
                # print('[DEBUG MASSAGE]')
                # for elem in graph.node[op_idx].attribute:
                #     print(elem)

                attri_nlist = []
                # get attribute name list
                for elem in graph.node[idx].attribute:
                    attri_nlist.append(elem.name)

                # find pads
                if 'pads' in attri_nlist:
                    idx1 = attri_nlist.index('pads')
                    temp_list.append(graph.node[idx].attribute[idx1].ints)
                else:
                    temp_list.append('None')

                # strides
                if 'strides' in attri_nlist:
                    idx1 = attri_nlist.index('strides')
                    temp_list.append(graph.node[idx].attribute[idx1].ints)
                else:
                    temp_list.append('None')
                temp_tuple = (graph.node[idx].name, temp_list[0], temp_list[1])
                print(temp_tuple)




        # attr=get_info.get_Gemm_attribute(graph)

        # init Gemm_attr
        list_Gemm_attr = []

        # traverse the Gemm operators and parse the attributeProtos
        if 'Gemm' not in idx_list.keys():
            print('[ERROR MASSAGE] This graph has no operators "Gemm" !')
            return False
        else:
            # traverse the all the Gemm operator
            for idx in idx_list['Gemm'].values():
                # traverse the attributes of a Gemm operator and get the name list
                attri_nlist = [k.name for k in graph.node[idx].attribute]
                # get transA
                if 'transA' in attri_nlist:
                    attr_idx = attri_nlist.index('transA')
                    transA = graph.node[idx].attribute[attr_idx].i
                else:
                    transA = 0
                # get transB
                if 'transB' in attri_nlist:
                    attr_idx = attri_nlist.index('transB')
                    transB = graph.node[idx].attribute[attr_idx].i
                else:
                    transB = 0
                # get alpha
                if 'alpha' in attri_nlist:
                    attr_idx = attri_nlist.index('alpha')
                    alpha = graph.node[idx].attribute[attr_idx].f
                else:
                    alpha = float(1.0)
                # get beta
                if 'beta' in attri_nlist:
                    attr_idx = attri_nlist.index('beta')
                    beta = graph.node[idx].attribute[attr_idx].f
                else:
                    beta = float(1.0)
                # collect the information
                temp_tuple = (graph.node[idx].name, transA, transB, alpha, beta)
                # append to the list
                list_Gemm_attr.append(temp_tuple)

        print(list_Gemm_attr)

    except Exception as e:
        print("Unable to display: " + str(e))
        return False

    return True
def get_valueproto_or_tensorproto_by_name(name: str, graph: xpb2.GraphProto):
    for i, node in enumerate(inferred_model.graph.node):
            if node.name == "":
                inferred_model.graph.node[i].name = str(i)
    input_nlist = [k.name for k in graph.input]
    initializer_nlist = [k.name for k in graph.initializer]
    value_info_nlist = [k.name for k in graph.value_info]
    output_nlist = [k.name for k in graph.output]

    # get tensor data
    if name in input_nlist:
        idx = input_nlist.index(name)
        return graph.input[idx], int(1)
    elif name in value_info_nlist:
        idx = value_info_nlist.index(name)
        return graph.value_info[idx], int(2)
    elif name in initializer_nlist:
        idx = initializer_nlist.index(name)
        return graph.initializer[idx], int(3)
    elif name in output_nlist:
        idx = output_nlist.index(name)
        return graph.output[idx], int(4)
    else:
        print("[ERROR MASSAGE] Can't find the tensor: ", name)
        print('input_nlist:\n', input_nlist)
        print('===================')
        print('value_info_nlist:\n', value_info_nlist)
        print('===================')
        print('initializer_nlist:\n', initializer_nlist)
        print('===================')
        print('output_nlist:\n', output_nlist)
        print('===================')
        return False, 0
def cal_tensor_mem_size(elem_type: str, shape: [int]):
    """ given the element type of the tensor and its shape, and return its memory size.

    Utility.

    Args:
        ttype: the type of the element of the given tensor. format: 'int', ...
        shape: the shape of the given tensor. format: [] of int

    Returns:
        mem_size: int
    """
    # init
    mem_size = int(1)
    # traverse the list to get the number of the elements
    for num in shape:
        mem_size *= num
    # multiple the size of variable with the number of the elements
    # "FLOAT": 1,
    # "UINT8": 2,
    # "INT8": 3,
    # "UINT16": 4,
    # "INT16": 5,
    # "INT32": 6,
    # "INT64": 7,
    # # "STRING" : 8,
    # "BOOL": 9,
    # "FLOAT16": 10,
    # "DOUBLE": 11,
    # "UINT32": 12,
    # "UINT64": 13,
    # "COMPLEX64": 14,
    # "COMPLEX128": 15
    if elem_type == 1:
        mem_size *= 4
    elif elem_type == 2:
        mem_size *= 1
    elif elem_type == 3:
        mem_size *= 1
    elif elem_type == 4:
        mem_size *= 2
    elif elem_type == 5:
        mem_size *= 2
    elif elem_type == 6:
        mem_size *= 4
    elif elem_type == 7:
        mem_size *= 8
    elif elem_type == 9:
        mem_size *= 1
    elif elem_type == 10:
        mem_size *= 2
    elif elem_type == 11:
        mem_size *= 8
    elif elem_type == 12:
        mem_size *= 4
    elif elem_type == 13:
        mem_size *= 8
    elif elem_type == 14:
        mem_size *= 8
    elif elem_type == 15:
        mem_size *= 16
    else:
        print("Undefined data type")

    return mem_size
def get_bandwidth(graph: xpb2.GraphProto):
    #try:
    mem_BW_list = []
    total_mem_BW = 0
    unknown_tensor_list = []
    # traverse all the nodes
    for nodeProto in graph.node:
        # init variables
        read_mem_BW_each_layer = 0
        write_mem_BW_each_layer = 0
        total_each_layer = 0
        # traverse all input tensor
        for input_name in nodeProto.input:
            # get the TensorProto/ValueInfoProto by searching its name
            proto, type_Num = get_valueproto_or_tensorproto_by_name(
                input_name, graph)
            # parse the ValueInfoProto/TensorProto
            if proto:
                if type_Num == 3:
                    dtype = getattr(proto, 'data_type', False)
                    # get the shape of the tensor
                    shape = getattr(proto, 'dims', [])
                elif type_Num == 1 or type_Num == 2:
                    name, dtype, shape_str = _parse_element_(proto)
                    if(len(shape_str)>2):#改了這裡
                        shape_str = shape_str.strip('[]')
                        shape_str = shape_str.split(',')
                        shape = []
                        for dim in shape_str:
                            shape.append(int(dim))
                else:
                    print(
                        '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                        input_name, ' is from a wrong list !')
            else:
                print(
                    '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                    input_name, ' is no found !')
                unknown_tensor_list.append(
                    (nodeProto.name, input_name, nodeProto.op_type))
            # calculate the tensor size in btye
            
            read_mem_BW_each_layer += cal_tensor_mem_size(dtype, shape)

        # traverse all output tensor
        for output_name in nodeProto.output:
            # get the TensorProto/ValueInfoProto by searching its name
            proto, type_Num = get_valueproto_or_tensorproto_by_name(
                output_name, graph)
            # parse the ValueInfoProto
            if proto:
                if type_Num == 2 or type_Num == 4:
                    # name, dtype, shape = utils._parse_ValueInfoProto(proto)
                    name, dtype, shape_str = _parse_element_(proto)
                    
                    if(len(shape_str)>2):#改了這裡
                        shape_str = shape_str.strip('[]')
                        shape_str = shape_str.split(',')
                        shape = []
                        for dim in shape_str:
                            shape.append(int(dim))
                else:
                    print(
                        '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                        output_name, ' is from a wrong list !')
            else:
                print(
                    '[ERROR MASSAGE] [get_info/mem_BW_without_buf] The Tensor: ',
                    input_name, ' is no found !')
                unknown_tensor_list.append(
                    (nodeProto.name, output_name, nodeProto.op_type))
            # calculate the tensor size in btye
            write_mem_BW_each_layer += cal_tensor_mem_size(dtype, shape)

        # cal total bw
        total_each_layer = read_mem_BW_each_layer + write_mem_BW_each_layer

        # store into tuple
        temp_tuple = (nodeProto.name, read_mem_BW_each_layer,
                    write_mem_BW_each_layer, total_each_layer)
        #append it
        mem_BW_list.append(temp_tuple)
        # accmulate the value
        total_mem_BW += total_each_layer

    # display the mem_bw of eahc layer
    columns = ['layer', 'read_bw', 'write_bw', 'total_bw']
    # resort the list
    mem_BW_list = sorted(mem_BW_list,
                            key=lambda Layer: Layer[1],
                            reverse=True)
    #print(tabulate(mem_BW_list, headers=columns))
    print(
        '====================================================================================\n'
    )
    # display it
    print("2-2-2. Data bandwidth requirement\n")
    
    print(
        "The memory bandwidth for processor to execute a whole model without on-chip-buffer is: \n",
        total_mem_BW, '(bytes)\n',
        float(total_mem_BW) / float(1000000), '(MB)\n')
    # display the unknown tensor
    columns = ['op_name', 'unfound_tensor', 'op_type']
    #print(tabulate(unknown_tensor_list, headers=columns))
    print(
        '====================================================================================\n'
    )
    # except Exception as e:
    #     print("[ERROR MASSAGE] Unable to display: " + str(e))
    #     return False

    return True


shape inference complete ...


In [7]:
def list_tensor_size(graph: xpb2.GraphProto):
    # main.List_Tensor_SizeOfConv(graph)
    try:
        for i, node in enumerate(inferred_model.graph.node):
                if node.name == "":
                    inferred_model.graph.node[i].name = str(i)
        # get the list
        All_Conv_tensor_size = []
        All_Gemm_tensor_size = []
        bias = False
        # get the idx of the operators
        if type(graph) is not xpb2.GraphProto:
            sys.exit('The input graph is not a GraphProto!')

        node_nList = [k.name for k in graph.node]
        input_nlist = [k.name for k in graph.input]
        initializer_nlist = [k.name for k in graph.initializer]
        value_info_nlist = [k.name for k in graph.value_info]
        output_nlist = [k.name for k in graph.output]
        idx_list = {}
        for node in graph.node:
            if node.op_type in idx_list:
                idx_list[node.op_type][node.name] = node_nList.index(node.name)
            else:
                idx_list[node.op_type] = {
                    node.name: node_nList.index(node.name)
                }

        # get the Conv tensor size
        if 'Conv' not in idx_list.keys():
            print('This graph has no operators "Conv" !')
        else:
            for idx in idx_list['Conv'].values():
                # temp_tuple, bias = utils._Cal_tensor_size_ConvOrGemm(idx, graph)
                num_conv_input_tensor = len(graph.node[idx].input)
                list_of_data_num = []
                # get input tensor proto
                for input_name in graph.node[idx].input:
                    # get tensor data
                    if input_name in input_nlist:
                        name_idx = input_nlist.index(input_name)
                        data = graph.input[name_idx]
                        type_num = int(1)
                    elif input_name in value_info_nlist:
                        name_idx = value_info_nlist.index(input_name)
                        data = graph.value_info[name_idx]
                        type_num = int(2)
                    elif input_name in initializer_nlist:
                        name_idx = initializer_nlist.index(input_name)
                        data = graph.initializer[name_idx]
                        type_num = int(3)
                    elif input_name in output_nlist:
                        name_idx = output_nlist.index(input_name)
                        data = graph.output[name_idx]
                        type_num = int(4)
                    else:
                        print("Can't find the tensor: ", input_name)
                        print('input_nlist:\n', input_nlist)
                        print('===================')
                        print('value_info_nlist:\n', value_info_nlist)
                        print('===================')
                        print('initializer_nlist:\n', initializer_nlist)
                        print('===================')
                        print('output_nlist:\n', output_nlist)
                        print('===================')
                
                    list_of_data_num.append((data, type_num))

                if graph.node[idx].output[0] in input_nlist:
                    name_idx = input_nlist.index(graph.node[idx].output[0])
                    data = graph.input[name_idx]
                    type_num = int(1)
                elif graph.node[idx].output[0] in value_info_nlist:
                    name_idx = value_info_nlist.index(graph.node[idx].output[0])
                    data = graph.value_info[name_idx]
                    type_num = int(2)
                elif graph.node[idx].output[0] in initializer_nlist:
                    name_idx = initializer_nlist.index(graph.node[idx].output[0])
                    data = graph.initializer[name_idx]
                    type_num = int(3)
                elif graph.node[idx].output[0] in output_nlist:
                    name_idx = output_nlist.index(graph.node[idx].output[0])
                    data = graph.output[name_idx]
                    type_num = int(4)
                else:
                    print("Can't find the tensor: ", graph.node[idx].output[0])
                    print('input_nlist:\n', input_nlist)
                    print('===================')
                    print('value_info_nlist:\n', value_info_nlist)
                    print('===================')
                    print('initializer_nlist:\n', initializer_nlist)
                    print('===================')
                    print('output_nlist:\n', output_nlist)
                    print('===================')
                list_of_data_num.append((data, type_num))

                list_temp = [
                    graph.node[idx].name,
                ]
                for elem in list_of_data_num:
                    if elem[0]:
                        if elem[1] == 3:
                            name = getattr(elem[0], 'name', "None")
                            # get the data type of the tensor
                            data_type = getattr(elem[0], 'data_type', False)
                            # get the shape of the tensor
                            shape = getattr(elem[0], 'dims', [])
                        else:
                            # name, data_type, shape = utils._parse_ValueInfoProto(elem[0])
                            name, data_type, shape_str = _parse_element_(elem[0])
                            if(len(shape_str)>2):
                                shape_str = shape_str.strip('[]')
                                shape_str = shape_str.split(',')
                                shape = []
                                for dim in shape_str:
                                    shape.append(int(dim))
                        mem_size = int(1)
                        # traverse the list to get the number of the elements
                        for num in shape:
                            mem_size *= num
                        # multiple the size of variable with the number of the elements
                        # "FLOAT": 1,
                        # "UINT8": 2,
                        # "INT8": 3,
                        # "UINT16": 4,
                        # "INT16": 5,
                        # "INT32": 6,
                        # "INT64": 7,
                        # # "STRING" : 8,
                        # "BOOL": 9,
                        # "FLOAT16": 10,
                        # "DOUBLE": 11,
                        # "UINT32": 12,
                        # "UINT64": 13,
                        # "COMPLEX64": 14,
                        # "COMPLEX128": 15
                        if data_type == 1:
                            mem_size *= 4
                        elif data_type == 2:
                            mem_size *= 1
                        elif data_type == 3:
                            mem_size *= 1
                        elif data_type == 4:
                            mem_size *= 2
                        elif data_type == 5:
                            mem_size *= 2
                        elif data_type == 6:
                            mem_size *= 4
                        elif data_type == 7:
                            mem_size *= 8
                        elif data_type == 9:
                            mem_size *= 1
                        elif data_type == 10:
                            mem_size *= 2
                        elif data_type == 11:
                            mem_size *= 8
                        elif data_type == 12:
                            mem_size *= 4
                        elif data_type == 13:
                            mem_size *= 8
                        elif data_type == 14:
                            mem_size *= 8
                        elif data_type == 15:
                            mem_size *= 16
                        list_temp.append(mem_size)
                    else:
                        print(graph.node[idx].name, 'tenosr no found ! Something wrong')

                if len(list_of_data_num) > 3:  # the conv has bias
                    ConvOrGemm_tensor_size = (list_temp[0], list_temp[1], list_temp[2],
                                            list_temp[3], list_temp[4], list_temp[1] +
                                            list_temp[2] + list_temp[3] + list_temp[4])
                    bias = True
                else:  # the conv has no bias
                    ConvOrGemm_tensor_size = (list_temp[0], list_temp[1], list_temp[2],
                                            list_temp[3],
                                            list_temp[1] + list_temp[2] + list_temp[3])
                    bias = False

                All_Conv_tensor_size.append(ConvOrGemm_tensor_size)
    # main.List_Tensor_SizeOfGemm(graph)
        if 'Gemm' not in idx_list.keys():
            print('This graph has no operators "Gemm" !')
        else:
            for idx in idx_list['Gemm'].values():
                num_conv_input_tensor = len(graph.node[idx].input)
                list_of_data_num = []
                for input_name in graph.node[idx].input:
                    if input_name in input_nlist:
                        name_idx = input_nlist.index(input_name)
                        data = graph.input[name_idx]
                        type_num = int(1)
                    elif input_name in value_info_nlist:
                        name_idx = value_info_nlist.index(input_name)
                        data = graph.value_info[name_idx]
                        type_num = int(2)
                    elif input_name in initializer_nlist:
                        name_idx = initializer_nlist.index(input_name)
                        data = graph.initializer[name_idx]
                        type_num = int(3)
                    elif input_name in output_nlist:
                        name_idx = output_nlist.index(input_name)
                        data = graph.output[name_idx]
                        type_num = int(4)
                    else:
                        print("Can't find the tensor: ", input_name)
                        print('input_nlist:\n', input_nlist)
                        print('===================')
                        print('value_info_nlist:\n', value_info_nlist)
                        print('===================')
                        print('initializer_nlist:\n', initializer_nlist)
                        print('===================')
                        print('output_nlist:\n', output_nlist)
                        print('===================')
                
                    list_of_data_num.append((data, type_num))
                if graph.node[idx].output[0] in input_nlist:
                    name_idx = input_nlist.index(graph.node[idx].output[0])
                    data = graph.input[name_idx]
                    type_num = int(1)
                elif graph.node[idx].output[0] in value_info_nlist:
                    name_idx = value_info_nlist.index(graph.node[idx].output[0])
                    data = graph.value_info[name_idx]
                    type_num = int(2)
                elif graph.node[idx].output[0] in initializer_nlist:
                    name_idx = initializer_nlist.index(graph.node[idx].output[0])
                    data = graph.initializer[name_idx]
                    type_num = int(3)
                elif graph.node[idx].output[0] in output_nlist:
                    name_idx = output_nlist.index(graph.node[idx].output[0])
                    data = graph.output[name_idx]
                    type_num = int(4)
                else:
                    print("Can't find the tensor: ", graph.node[idx].output[0])
                    print('input_nlist:\n', input_nlist)
                    print('===================')
                    print('value_info_nlist:\n', value_info_nlist)
                    print('===================')
                    print('initializer_nlist:\n', initializer_nlist)
                    print('===================')
                    print('output_nlist:\n', output_nlist)
                    print('===================')
                list_of_data_num.append((data, type_num))
                list_temp = [
                    graph.node[idx].name,
                ]
                for elem in list_of_data_num:
                    if elem[0]:
                        if elem[1] == 3:
                            name = getattr(elem[0], 'name', "None")
                            data_type = getattr(elem[0], 'data_type', False)
                            shape = getattr(elem[0], 'dims', [])
                        else:
                            name, data_type, shape_str = _parse_element_(elem[0])
                            if(len(shape_str)>2):
                                shape_str = shape_str.strip('[]')
                                shape_str = shape_str.split(',')
                                shape = []
                                for dim in shape_str:
                                    shape.append(int(dim))
                        mem_size = int(1)
                        # traverse the list to get the number of the elements
                        for num in shape:
                            mem_size *= num
                        # multiple the size of variable with the number of the elements
                        if data_type == 'FLOAT':
                            mem_size *= 8
                        elif data_type == 'UINT8':
                            mem_size *= 1
                        elif data_type == 'INT8':
                            mem_size *= 1
                        elif data_type == 'UINT16':
                            mem_size *= 2
                        elif data_type == 'INT16':
                            mem_size *= 2
                        elif data_type == 'INT32':
                            mem_size *= 4
                        elif data_type == 'INT64':
                            mem_size *= 8
                        elif data_type == 'BOOL':
                            mem_size *= 1
                        elif data_type == 'FLOAT16':
                            mem_size *= 2
                        elif data_type == 'DOUBLE':
                            mem_size *= 8
                        elif data_type == 'UINT32':
                            mem_size *= 4
                        elif data_type == 'UINT64':
                            mem_size *= 8
                        elif data_type == 'COMPLEX64':
                            mem_size *= 8
                        elif data_type == 'COMPLEX128':
                            mem_size *= 16
                        else:
                            print("Undefined data type")
                        list_temp.append(mem_size)
                    else:
                        print(graph.node[idx].name, 'tenosr no found ! Something wrong')

                if len(list_of_data_num) > 3:  # the conv has bias
                    ConvOrGemm_tensor_size = (list_temp[0], list_temp[1], list_temp[2],
                                            list_temp[3], list_temp[4], list_temp[1] +
                                            list_temp[2] + list_temp[3] + list_temp[4])
                    bias = True
                else:  # the conv has no bias
                    ConvOrGemm_tensor_size = (list_temp[0], list_temp[1], list_temp[2],
                                            list_temp[3],
                                            list_temp[1] + list_temp[2] + list_temp[3])
                    bias = False

                All_Gemm_tensor_size.append(ConvOrGemm_tensor_size)  
        # print
        print("2-2-3. activation memory storage requirement\n")
        if 'Conv' not in idx_list.keys():
            print('returning from List_Tensor_SizeOfConv() ...')
        else:
            if bias:
                columns = [
                    'Conv_name', 'Input_tesnor', 'Weight_tensor',
                    'Bias_tensor', 'Output_tensor', 'Total'
                ]
            else:
                columns = [
                    'Conv_name', 'Input_tesnor', 'Weight_tensor',
                    'Output_tensor', 'Total'
                ]

            # resort the list
            All_Conv_tensor_size = sorted(All_Conv_tensor_size,
                                          key=lambda Layer: Layer[-1],
                                          reverse=True)
            print('list_of_layer RESORTED ! ')
            print(tabulate(All_Conv_tensor_size, headers=columns))
            print(
                '====================================================================================\n'
            )

        if 'Gemm' not in idx_list.keys():
            print('returning from List_Tensor_SizeOfGemm() ...')
        else:
            if bias:
                columns = [
                    'Gemm_name', 'Mat_A', 'Mat_B', 'Mat_C', 'Mat_Y', 'Total'
                ]
            else:
                columns = ['Gemm_name', 'Mat_A', 'Mat_B', 'Mat_Y', 'Total']

            # resort the list
            All_Gemm_tensor_size = sorted(All_Gemm_tensor_size,
                                          key=lambda Layer: Layer[-1],
                                          reverse=True)
            print('list_of_layer RESORTED ! ')
            print(tabulate(All_Gemm_tensor_size, headers=columns))
            print(
                '====================================================================================\n'
            )

    except Exception as e:
        print("Unable to display: " + str(e))

In [8]:
#從這裡開始
print("start")
# find_operator(inferred_model.graph)#ok
# get_attribute(inferred_model.graph)#ok
get_bandwidth(inferred_model.graph)
list_tensor_size(inferred_model.graph)

start

2-2-2. Data bandwidth requirement

The memory bandwidth for processor to execute a whole model without on-chip-buffer is: 
 13951528 (bytes)
 13.951528 (MB)


Undefined data type
Undefined data type
Undefined data type
Undefined data type
2-2-3. activation memory storage requirement

list_of_layer RESORTED ! 
Conv_name      Input_tesnor    Weight_tensor    Bias_tensor    Output_tensor    Total
-----------  --------------  ---------------  -------------  ---------------  -------
Conv_95                   0          1638400           5120                0  1643520
Conv_94                   0          1228800           1280                0  1230080
Conv_78                   0           614400           3840                0   618240
Conv_84                   0           614400           3840                0   618240
Conv_90                   0           614400           3840                0   618240
Conv_82                   0           614400            640                0   6

Total activation memory: 0
